In [ ]:
'''
作成日：2025年10月20日
完了日:2025年10月25日
作成者：大田原丈瑠
'''
#インポート
#グローバル変数の宣言
ELEMENT_SYMBOLS = {"火":"$", "水":"~", "風":"@", "土":"#", "命":"&", "無":" ", }
ELEMENT_COLORS = {"火":41, "水":44, "風":42, "土":43, "命":45, "無":47, }
ELEMENT_BOOST = {("水", "火"):20, ("火", "風"):20, ("風","土"):20, ("土","水"):20, ("火", "水"):0.5, ("風", "火"):0.5, ("土", "風"):0.5, ("水", "土"):0.5}
#関数宣言
def main():
    error1 = True
    while error1:
        name = input("プレイヤー名を入力してください＞＞")
        if name !="":
            error1 = False
        else:
            print("エラー:プレイヤーの名を入力してください")
    print('*** puzzle & Monsters ***')
    azure_ragon = {"name":"青龍", "hp":150, "max_hp":150,
            "element":"風", "ap":15, "dp":10}
    vermilion_bird = {"name":"朱雀", "hp":150, "max_hp":150,
            "element":"火", "ap":25, "dp":10}
    white_tiger = {"name":"白虎", "hp":150, "max_hp":150,
            "element":"土", "ap":20, "dp":5}
    genbu = {"name":"玄武", "hp":150, "max_hp":150,
            "element":"水", "ap":20, "dp":15}
    friends = [azure_ragon, vermilion_bird, white_tiger, genbu]
    
    party = organize_party(name, friends)
    
    slime = {"name":"スライム", "hp":100, "max_hp":100,
            "element":"水", "ap":10, "dp":1}
    goblin = {"name":"ゴブリン", "hp":200, "max_hp":200,
            "element":"土", "ap":20, "dp":5}
    giantbat = {"name":"オオコウモリ", "hp":300, "max_hp":300,
            "element":"風", "ap":30, "dp":10}
    werewolf = {"name":"ウェアウルフ", "hp":400, "max_hp":400,
            "element":"風", "ap":40, "dp":15}
    dragon = {"name":"ドラゴン", "hp":600, "max_hp":600,
            "element":"火", "ap":50, "dp":20}
    monters = [slime, goblin, giantbat, werewolf, dragon]
    
    beet_count = go_dungeon(party, monters)
    if beet_count < 5:
        print(f"*** GAME OVER!! ***\n倒したモンスター数{beet_count}")
    else:
        print(f"*** GAME CLEARED ***\n倒したモンスター数{beet_count}")
    
def go_dungeon(party, monsters):
    party_leader = "name"
    party_hp = "hp"
    print(f"{party[party_leader]}のパーティーhp={party[party_hp]}はダンジョンに到着した")
    print("<パーティー編成>----------------")
    print(show_party(party))
    print("-------------------------------\n")
    is_win = 0
    for monster in monsters:
        is_win += do_battle(party, monster)
        total_hp = party["hp"]
        if total_hp <= 0:
            print(f"{party[party_leader]}はダンジョンから逃げ出した")
            break
        else:
            print(f"{party[party_leader]}はさらに奥へと進んだ\n===============================")
    if is_win ==5:
        print(f"{party[party_leader]}はダンジョンを制覇した")
    return  is_win #倒したモンスターの数

def do_battle(party, monster):
    print(f"{print_monster_name(monster)}が現れた！")
    jewel_list = fill_gams()
    while party["hp"] > 0 and monster["hp"] > 0:
        party, monster, jewel_list = on_player_turn(party, monster, jewel_list)
        if monster["hp"] <=0:
            break
        else:
            on_enemy_turn(party, monster)
    if party["hp"] <= 0:
        print("パーティーのHPは0になった")
        return 0
    else:
        print(f"{print_monster_name(monster)}を倒した！")
        return 1

def on_player_turn(party, monster, jewel_list):
    party_leader = "name"
    party_hp = "hp"
    print(f"【{party[party_leader]}のターン】(HP={party[party_hp]})")
    show_battle_field(party, monster, jewel_list)
    command = input("コマンド?>")
    jewel_list = move_gem(command, jewel_list)
    party, jewel_list, monster = evaluate_gems(party, jewel_list, monster)
    return party, monster, jewel_list

def on_enemy_turn(party, monster):
    monster_name = "name"
    monster_hp = "hp"
    print(f"【{monster[monster_name]}のターン】(HP={monster[monster_hp]})")
    party = do_enemy_attack(party, monster)
    return party

def do_attack(party, monster, element, combo_correction, combo):
    if element == "命":
        recover = do_recover(combo_correction)
        if type(recover) == float:
         recover = int(recover)
        party["hp"] += recover
        if party["hp"] > party["max_hp"]:
            party["hp"] = party["max_hp"]
            recover - (party["hp"]-party["max_hp"])
        print(f"パーティは{recover}回復した")
    elif element != 0:
        normol_attack, buddy = blur_damage(element, combo_correction, party, monster)
        if type(normol_attack) == float:
            normol_attack = int(normol_attack)
        buddy_name = print_monster_name(buddy)
        if combo > 1:
            print(f"{buddy_name}の攻撃！ {combo} Commbo!!")
        else:
            print(f"{buddy_name}の攻撃！")
        print(f"敵モンスターに{normol_attack}のダメージを与えた")
        monster["hp"] -= normol_attack
    else:
        pass
    return party, monster

def do_enemy_attack(party, monster):
    pre_attack = monster["ap"] - party["dp"]
    import random
    attack = pre_attack + pre_attack * random.uniform(-0.1,0.1)
    if type(attack) == float:
            attack = int(attack)
    if attack < 0:
        attack = 1
    print(f"パーティに{attack}のダメージを与えた\n")
    party["hp"] -= attack
    return party
    
def print_monster_name(monster):
    monster_name = monster["name"]
    monster_element = monster["element"]
    color = ELEMENT_COLORS[monster_element]
    return f"\033[{color}m{ELEMENT_SYMBOLS[monster_element]}{monster_name}{ELEMENT_SYMBOLS[monster_element]}\033[0m"

def organize_party(name, friends):
    name_palyer = name
    total_hp = 0
    total_dp = 0
    for friend in friends:
        total_hp += friend["hp"]
        total_dp += friend["dp"]
    ave_protect = total_dp / len(friends)
    max_hp = total_hp
    party = {"name":name_palyer, "friends":friends, "hp":total_hp,
            "max_hp":max_hp, "dp":ave_protect}
    return party

def show_party(party):
    friends = party["friends"]
    friends_easy = ""
    party_terms = {"HP=":"hp", "攻撃":"ap", "防御":"dp"}
    for friend in friends:
        friend_easy = print_monster_name(friend)
        for party_term in party_terms:
            friend_easy += (party_term + str(friend[party_terms[party_term]]))
        if friend == friends[0]:
            friends_easy += friend_easy
        else:
            friends_easy += ("\n" + friend_easy)
    return friends_easy

def make_alphabet(a = "A", b = "N"):
    start_code = ord(a)
    end_code = ord(b)
    if start_code > end_code:
        alphabet_list = [chr(i) for i in range(start_code, end_code -1, -1)]
    else:
        alphabet_list = [chr(i) for i in range(start_code, end_code + 1)]
    return alphabet_list

def fill_gams():
    jewel_list = list()
    from random import choice
    symbol_list = [{key: value} for key, value in ELEMENT_SYMBOLS.items()]
    #[{'火': '$'}, {'水': '~'}, {'風': '@'}, {'土': '#'}, {'命': '&'}, {'無': ''}]
    symbol_list.remove({"無":" "})
    for alphabet in range(14):
        jewel_list.append(choice(symbol_list))
    return jewel_list

def print_gems(jewel_list):
    print("--------------------------------")
    keys_jewel = ' '.join(make_alphabet())
    print(keys_jewel)
    for value_jewel in jewel_list:
        color_key1, symbol_key = list(value_jewel.items())[0]
        color_key = ELEMENT_COLORS[color_key1]
        print(f"\033[{color_key}m{symbol_key}\033[0m", end=" ")
    print("\n--------------------------------")

def show_battle_field(party, monster, jewel_list):
    print("バトルフィールド")
    monster_hp = monster["hp"]
    monster_hpmax = monster["max_hp"]
    print(f"{print_monster_name(monster)}HP = {monster_hp} / {monster_hpmax}\n")
    for friend in party["friends"]:
        print(print_monster_name(friend), end="")
    party_hp = party["hp"]
    party_hpmax = party["max_hp"]
    print(f"\nHP = {party_hp} / {party_hpmax}")
    print_gems(jewel_list)

def check_valid_command(command):
    alphabet_list =make_alphabet()
    check = False
    while check == False:
        alphabet_set = set(alphabet_list)
        command_sets = set(command)
        same_ABC = alphabet_set & command_sets
        if len(command_sets) == 2 and len(same_ABC) == 2:
            check = True
        else:
            command = input("値が正しくありません。もう一度A~Nまで入力してくだい")
    return list(command)

def move_gem(command, jewel_list):
    alphabet_list = make_alphabet()
    command_list = check_valid_command(command)
    start_command = alphabet_list.index(command_list[0])
    end_command = alphabet_list.index(command_list[1])
    gems_show(jewel_list)
    if start_command < end_command:
        for num in range(start_command, end_command):
            jewel_list = switch_gem(num, num+1, jewel_list)
    else:
        for num in range(start_command, end_command, -1):
            jewel_list = switch_gem(num, num-1, jewel_list)
    return jewel_list

def switch_gem(num, num2, jewel_list):
    switch1 = jewel_list[num].copy()
    switch2 = jewel_list[num2].copy()
    jewel_list[num2] = switch1
    jewel_list[num] = switch2
    gems_show(jewel_list)
    return jewel_list

def check_banishable(jewel_list):
    banish_list = []
    total = len(jewel_list)
    for i in range(total-2):
        if jewel_list[i] == jewel_list[i+1] == jewel_list[i+2]:
            current_jewel = jewel_list[i]
            end_i = i + 2
            for j in range(i + 3, total):
                if jewel_list[j] == current_jewel:
                    end_i = j
                else:
                    break
            banish_list = list(range(i, end_i+1))
            break
    return banish_list

def banish_gems(jewel_list, banish_list):
    effect_count = list()
    effect_element = list()
    if len(banish_list) != 0:
        effect_count.append(len(banish_list))
        effect_element.append(jewel_list[banish_list[0]])
        for banish_index in banish_list:
            jewel_list[banish_index] = {"無":" "}
    if len(effect_count) >= 1:
        gems_show(jewel_list)
        effect_list = [effect_count[0], effect_element[0]]
    else:
        effect_list = list()
    return jewel_list, effect_list

def shift_gems(jewel_list):
    check = True
    while check:
        if not {"無":" "} in jewel_list:
            check = False
        else:
            index_0 = jewel_list.index({"無":" "})
            for i in range(index_0+1, 14):
                if jewel_list[i] == {"無":" "} and i != 13:
                    continue
                elif i != 13:
                    jewel_list = switch_gem(index_0, i, jewel_list)
                    break
                elif i == 13 and jewel_list[i] == {"無":" "}:
                    check = False
                else:
                    jewel_list = switch_gem(index_0, i, jewel_list)
                    check = False
    return jewel_list

def gems_show(jewel_list):
    count = 0
    for value_jewel in jewel_list:
        color_key1, symbol_key = list(value_jewel.items())[0]
        color_key = ELEMENT_COLORS[color_key1]
        if count != 0 and count % 13 == 0:
            print(f"\033[{color_key}m{symbol_key}\033[0m")
        else:
            count += 1
            print(f"\033[{color_key}m{symbol_key}\033[0m", end=" ")

def spawn_gems(jewel_list):
    from random import choice
    symbol_list = [{key: value} for key, value in ELEMENT_SYMBOLS.items()]
    #[{'火': '$'}, {'水': '~'}, {'風': '@'}, {'土': '#'}, {'命': '&'}, {'無': ''}]
    symbol_list.remove({"無":" "})
    count = 0
    for i in range(len(jewel_list)-1, -1, -1):
        if jewel_list[i] == {"無":" "}:
            jewel_list[i] = choice(symbol_list)
            count += 1
        else:
            break
    if count >= 1:
        gems_show(jewel_list)
    return jewel_list

def evaluate_gems(party, jewel_list, monster):
    count = 1
    combo = 0
    while count == 1:
        banish_list = check_banishable(jewel_list)
        if banish_list == list():
            count = 0
        else:
            jewel_list, effect_list = banish_gems(jewel_list, banish_list)
            combo += 1
            element, delete_gem = check_element(effect_list)
            combo_correction = 1.5 ** (delete_gem - 3 + combo)
            party, monster = do_attack(party, monster, element, combo_correction, combo)
            jewel_list = shift_gems(jewel_list)
            jewel_list = spawn_gems(jewel_list)
    print("")
    return party, jewel_list, monster

def check_element(effect_list):
    element = list(effect_list[1].keys())[0]
    delete_gem = effect_list[0]
    return element, delete_gem

def blur_damage(element, combo_correction, party, monster):
    for friend in party["friends"]:
        if element in friend.values():
            friend_index = party["friends"].index(friend)
            break
    buddy = party["friends"][friend_index].copy()
    if (element, monster["element"]) in ELEMENT_BOOST:
        boost = ELEMENT_BOOST[(element, monster["element"])]
    else:
        boost = 1.0
    pre_normol_attack = (buddy["ap"] - monster["dp"]) * boost * combo_correction
    import random
    normol_attack = pre_normol_attack + pre_normol_attack * random.uniform(-0.1,0.1)
    if normol_attack < 0:
        normol_attack = 1
    return normol_attack, buddy

def do_recover(combo_correction):
    import random
    recover_num = (20 * combo_correction)
    recover = recover_num + recover_num * random.uniform(-0.1,0.1)
    return recover

# main関数の呼び出し
main()
